In [1]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

get_ipython().run_line_magic('matplotlib', 'inline')

import math
import random
from datetime import timedelta

import warnings 
warnings.filterwarnings('ignore')
#color pallette
cnf = '#393e46'
dth = '#ff2e63'
rec = '#21bf73'
act = '#fe9801'

In [2]:
import plotly as py
py.offline.init_notebook_mode(connected = True)

In [4]:
import os





try:
    os.system("rm -rf Covid-19-Preprocessed-Dataset")
except:
    print("File doesnot exsist")





In [5]:
get_ipython().system('git clone https://github.com/laxmimerit/Covid-19-Preprocessed-Dataset.git')

'git' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
df = pd.read_csv('Desktop\Covid-19-Preprocessed-Dataset\preprocessed\covid_19_data_cleaned.csv', parse_dates=['Date'])
country_daywise = pd.read_csv('Desktop\Covid-19-Preprocessed-Dataset\preprocessed\country_daywise.csv', parse_dates=['Date'])
countrywise = pd.read_csv('Desktop\Covid-19-Preprocessed-Dataset\preprocessed\countrywise.csv')
daywise = pd.read_csv('Desktop\Covid-19-Preprocessed-Dataset\preprocessed\daywise.csv', parse_dates=['Date'])




df['Province/State'] = df['Province/State'].fillna("")
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'Desktop\\Covid-19-Preprocessed-Dataset\\preprocessed\\covid_19_data_cleaned.csv'

In [ ]:

confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()
recovered = df.groupby('Date').sum()['Recovered'].reset_index()
deaths = df.groupby('Date').sum()['Deaths'].reset_index()
deaths.head()


In [ ]:
df.isnull().sum()

In [ ]:
df.query('Country == "US"')

In [ ]:
confirmed.tail()

In [ ]:
recovered.tail()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = confirmed['Date'], y=confirmed['Confirmed'], mode = 'lines+markers', name = 'Confirmed', line = dict(color = "Orange", width = 4)))
fig.add_trace(go.Scatter(x = recovered['Date'], y=recovered['Recovered'], mode = 'lines+markers', name = 'Recovered', line = dict(color = "Green", width = 4)))
fig.add_trace(go.Scatter(x = deaths['Date'], y=deaths['Deaths'], mode = 'lines+markers', name = 'Deaths', line = dict(color = "Red", width = 4)))
fig.update_layout(title = 'Worldwide Covid-19 Cases', xaxis_tickfont_size = 14, yaxis=dict(title = 'No of cases'))
fig.show()


In [ ]:
df['Date'] = df['Date'].astype(str)





df.info()



In [ ]:
fig = px.density_mapbox(df, lat = 'Lat', lon = 'Long', hover_name = 'Country', hover_data = ['Confirmed' ,'Recovered', 'Deaths'], animation_frame='Date', color_continuous_scale='Portland', radius = 7,zoom = 0,height = 700)
fig.update_layout(title = 'Worldwide Covid-19 cases with timelapse')
fig.update_layout(mapbox_style = 'open-street-map', mapbox_center_lon = 0)
fig.show()

In [ ]:
temp =df.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop = True)
tm = temp.melt(id_vars = 'Date' , value_vars = ['Active', 'Deaths', 'Recovered'])
fig = px.treemap(tm, path = ['variable'], values = 'value', height = 250, width = 800,color_discrete_sequence=[act,rec,dth])

fig.data[0].textinfo = 'label+text+value'
fig.show()


In [ ]:
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars = 'Date', value_vars = ['Recovered', 'Deaths','Active'], var_name = 'Case',value_name = 'Count')
fig = px.area(temp, x = 'Date', y = 'Count', color = 'Case', height = 600, title = 'Cases over time',color_discrete_sequence=[rec, dth, act])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()


In [ ]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron', min_zoom=1, max_zoom=4, zoom_start=1)
for i in range(0, len(temp)):
    
    folium.Circle(
    location=[full_latest.iloc[i]['Lat'], full_latest.iloc[i]['Long']],
    color='crimson',
    tooltip = 'Country : '+str(full_latest.iloc[i]['Country/Region'])+
              'Province : '+str(full_latest.iloc[i]['Province/State'])+
              'Confirmed : '+str(full_latest.iloc[i]['Confirmed'])+
              'Deaths : '+str(full_latest.iloc[i]['Deaths'])+
              'Recovered : '+str(full_latest.iloc[i]['Recovered']),
    radius=int(full_latest.iloc[i]['Confirmed'])**1.1).add_to(m)

# confirmed case

In [ ]:
fig = px.choropleth(country_daywise,locations = 'Country',locationmode = 'country names',color = country_daywise['Confirmed'],
                    hover_name = 'Country',animation_frame = country_daywise['Date'].dt.strftime('%Y-%m-%d'),
                    title = 'Cases over time',color_continuous_scale = px.colors.sequential.Inferno)
fig.update(layout_coloraxis_showscale = True)
fig.show()

# Death and recover per 100 cases 

In [ ]:
fig_c = px.bar(daywise , x = 'Date' , y = 'Confirmed' , color_discrete_sequence=[act])
fig_d = px.bar(daywise , x = 'Date' , y = 'Deaths' , color_discrete_sequence=['aqua'])

fig = make_subplots(rows = 1,cols = 2, shared_xaxes=False, horizontal_spacing=0.1,
                   subplot_titles=("Confirmed Cases" , "Death Cases"))
fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_c['data'][0], row = 1, col = 2)
fig.update_layout(height = 400)
fig.show()

# confirmed and death cases with static colormap

In [ ]:
fig_c = px.choropleth(countrywise,locations='Country',locationmode='country names',
                     color = np.log(countrywise['Confirmed']),hover_name='Country',
                     hover_data = ['Confirmed'])
temp = countrywise[countrywise['Deaths']>0]
fig_d = px.choropleth(temp,locations='Country',locationmode='country names',
                     color = np.log(temp['Deaths']),hover_name='Country',
                     hover_data = ['Deaths'])
fig = make_subplots(rows = 1,cols = 2,subplot_titles = ['Confirmed','Deaths'],
                    specs =[[{'type':'choropleth'},{'type':'choropleth'}]])

fig.add_trace(fig_c['data'][0],row = 1, col = 1)
fig.add_trace(fig_d['data'][0],row = 1, col = 2)
fig.show()
                    
                



In [ ]:
fig1 = px.line(daywise, x ='Date', y = 'Deaths / 100 Cases',color_discrete_sequence=[dth])
fig2 = px.line(daywise, x ='Date', y = 'Recovered / 100 Cases',color_discrete_sequence=[rec])
fig3 = px.line(daywise, x ='Date', y = 'Deaths / 100 Recovered',color_discrete_sequence=['aqua'])

fig = make_subplots(rows = 1,cols = 3, shared_xaxes= False,
                   subplot_titles=('Deaths / 100 Cases','Recovered / 100 Cases','Deaths / 100 Recovered'))
fig.add_trace(fig1['data'][0],row = 1, col = 1)
fig.add_trace(fig2['data'][0],row = 1, col = 2)
fig.add_trace(fig3['data'][0],row = 1, col = 3)

fig.update_layout(height = 400)
fig.show()

In [ ]:
daywise.columns

# New cases and no of countries

In [ ]:
fig_c = px.bar(daywise, x= 'Date', y = 'Confirmed', color_discrete_sequence=[cnf])
fig_d = px.bar(daywise, x= 'Date', y = 'No. of Countries', color_discrete_sequence=[dth])

fig = make_subplots(rows = 1,cols = 3, shared_xaxes= False,horizontal_spacing=0.1,
                    subplot_titles=("No, of New Cases per Day","No. of Countries"))
fig.add_trace(fig_c['data'][0], row = 1,col = 1)
fig.add_trace(fig_d['data'][0], row = 1,col = 2)

fig.show()


# Top 15 countries cases Analysis

In [ ]:
top = 15
fig_c = px.bar(countrywise.sort_values('Confirmed').tail(top),x='Confirmed',y='Country',
              text = 'Confirmed',orientation='h',color_discrete_sequence=[cnf])
fig_d = px.bar(countrywise.sort_values('Deaths').tail(top),x='Deaths',y='Country',
              text = 'Deaths',orientation='h',color_discrete_sequence=[dth])

fig_a = px.bar(countrywise.sort_values('Active').tail(top),x='Active',y='Country',
              text = 'Active',orientation='h',color_discrete_sequence=[act])
fig_r = px.bar(countrywise.sort_values('Recovered').tail(top),x='Recovered',y='Country',
              text = 'Recovered',orientation='h',color_discrete_sequence=[rec])
fig_dc = px.bar(countrywise.sort_values('Deaths / 100 Cases').tail(top),x='Deaths / 100 Cases',y='Country',
              text = 'Deaths / 100 Cases',orientation='h',color_discrete_sequence=['#f84351'])
fig_rc = px.bar(countrywise.sort_values('Recovered / 100 Cases').tail(top),x='Recovered / 100 Cases',y='Country',
              text = 'Recovered / 100 Cases',orientation='h',color_discrete_sequence=['#a45398'])




fig = make_subplots(rows = 5,cols = 2, shared_xaxes= False,horizontal_spacing=0.2,
                    vertical_spacing=0.05,
                    subplot_titles=('Confirmed Cases','Deaths Reported','Active Cases','Recovered Cases','Deaths / 100 Cases','Recovered / 100 Cases'))

fig.add_trace(fig_c['data'][0], row = 1,col = 1)
fig.add_trace(fig_d['data'][0], row = 1,col = 2)
fig.add_trace(fig_a['data'][0], row = 2,col = 1)
fig.add_trace(fig_r['data'][0], row = 2,col = 2)
fig.add_trace(fig_dc['data'][0], row = 3,col = 1)
fig.add_trace(fig_rc['data'][0], row = 3,col = 2)
fig.update_layout(height=4000)

fig.show()



In [ ]:
countrywise.columns

In [ ]:
#conda install -c plotly plotly-orca==1.2.1 psutil requests

In [ ]:
if not os.path.exists('images'):
    os.mkdir('images')

In [ ]:
fig.write_image('images/fig.png')

# Scatter plot for deaths Vs Confirmed Cases

In [ ]:
top = 15
fig = px.scatter(countrywise.sort_values('Deaths',ascending = False).head(top),
                x = 'Confirmed', y = 'Deaths', color = 'Country', size = 'Confirmed',height = 700,
                text = 'Country',log_x = True, log_y = True , title='Deaths vs Confirmed Cases(Cases are on log 10 scale)')
fig.update_traces(textposition = 'top center')
fig.update_layout(showlegend = False)
fig.update_layout(xaxis_rangeslider_visible = True)
fig.show()

In [ ]:
from IPython.display import display
from ipywidgets import *
button = widgets.Button(description="Click Me")
display(button)
def on_button_clicked(a):
    display(HTML("""<a href="http://localhost:8888/view/images/fig.png">CLICK ME</a>"""))
button.on_click(on_button_clicked)